In [1]:
!pip install fastapi uvicorn sqlalchemy pydantic pytest nest-asyncio



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\my pc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import sqlite3
import time
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
import uvicorn


In [3]:
conn = sqlite3.connect("edtech.db", check_same_thread=False)
cursor = conn.cursor()

cursor.executescript("""
DROP TABLE IF EXISTS students;
DROP TABLE IF EXISTS courses;
DROP TABLE IF EXISTS enrollments;

CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    grade TEXT,
    created_at DATE
);

CREATE TABLE courses (
    id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT
);

CREATE TABLE enrollments (
    id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    enrolled_at DATE
);
""")

students = [
    ("Alice", "A", "2024-01-10"),
    ("Bob", "B", "2024-02-12"),
    ("Charlie", "A", "2023-11-05"),
    ("David", "C", "2024-03-15"),
    ("Eva", "B", "2024-04-20"),
    ("Frank", "A", "2024-05-22"),
    ("Grace", "A", "2023-10-10"),
    ("Helen", "B", "2024-02-18"),
    ("Ian", "C", "2024-06-01"),
    ("Jane", "A", "2024-07-11")
]

courses = [
    ("Python Basics", "Programming"),
    ("Advanced Python", "Programming"),
    ("Math Fundamentals", "Math"),
    ("Data Science", "AI"),
    ("Machine Learning", "AI")
]

cursor.executemany(
    "INSERT INTO students(name, grade, created_at) VALUES (?, ?, ?)",
    students
)

cursor.executemany(
    "INSERT INTO courses(name, category) VALUES (?, ?)",
    courses
)

enrollments = []
eid = 1
for s_id in range(1, 11):
    for c_id in range(1, 3):
        enrollments.append((eid, s_id, c_id, "2024-03-01"))
        eid += 1

cursor.executemany(
    "INSERT INTO enrollments VALUES (?, ?, ?, ?)",
    enrollments
)

conn.commit()
print("Database seeded successfully")


Database seeded successfully


In [4]:
def generate_sql(question: str) -> str:
    q = question.lower()

    if "how many" in q and "python" in q and "2024" in q:
        return """
        SELECT COUNT(DISTINCT students.id)
        FROM students
        JOIN enrollments ON students.id = enrollments.student_id
        JOIN courses ON courses.id = enrollments.course_id
        WHERE courses.name LIKE '%Python%'
        AND enrollments.enrolled_at BETWEEN '2024-01-01' AND '2024-12-31'
        """

    if "total students" in q:
        return "SELECT COUNT(*) FROM students"

    raise ValueError("Unable to generate SQL for the given question")


In [5]:
def validate_sql(sql: str):
    forbidden = ["DROP", "DELETE", "UPDATE", "INSERT"]
    for word in forbidden:
        if word in sql.upper():
            raise ValueError("Unsafe SQL detected")


In [6]:
def execute_sql(sql: str):
    validate_sql(sql)
    cursor = conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()


In [7]:
analytics = {
    "total_queries": 0,
    "keywords": {},
    "slowest_ms": 0
}

def update_analytics(question, exec_time):
    analytics["total_queries"] += 1

    for word in question.lower().split():
        analytics["keywords"][word] = analytics["keywords"].get(word, 0) + 1

    analytics["slowest_ms"] = max(analytics["slowest_ms"], exec_time)


In [8]:
app = FastAPI()

class QueryRequest(BaseModel):
    question: str

@app.post("/query")
def query_db(req: QueryRequest):
    start = time.time()

    sql = generate_sql(req.question)
    result = execute_sql(sql)

    exec_time = round((time.time() - start) * 1000, 2)
    update_analytics(req.question, exec_time)

    return {
        "generated_sql": sql.strip(),
        "result": result,
        "execution_time_ms": exec_time
    }

@app.get("/stats")
def get_stats():
    return {
        "total_queries": analytics["total_queries"],
        "common_keywords": sorted(
            analytics["keywords"],
            key=analytics["keywords"].get,
            reverse=True
        ),
        "slowest_query_ms": analytics["slowest_ms"]
    }


In [9]:
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [32452]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50511 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50511 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57602 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57602 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61026 - "POST /query HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [32452]
